In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-12-20 08:07:33--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  71.6MB/s    in 0.3s    

2024-12-20 08:07:34 (71.6 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# add your code here - consider creating a new cell for each section of code

In [5]:
#Merge df_books and df_ratings
df = df_ratings.merge(df_books, on='isbn',how="left")


In [6]:
df.head

<bound method NDFrame.head of            user         isbn  rating  \
0        276725   034545104X     0.0   
1        276726   0155061224     5.0   
2        276727   0446520802     0.0   
3        276729   052165615X     3.0   
4        276729   0521795028     6.0   
...         ...          ...     ...   
1149775  276704   1563526298     9.0   
1149776  276706   0679447156     0.0   
1149777  276709   0515107662    10.0   
1149778  276721   0590442449    10.0   
1149779  276723  05162443314     8.0   

                                                     title             author  
0                                     Flesh Tones: A Novel         M. J. Rose  
1                                         Rites of Passage         Judith Rae  
2                                             The Notebook    Nicholas Sparks  
3                                           Help!: Level 1      Philip Prowse  
4        The Amsterdam Connection : Level 4 (Cambridge ...        Sue Leather  
...                                                    ...                ...  
1149775  Get Clark Smart : The Ultimate Guide for the S...       Clark Howard  
1149776  Eight Weeks to Optimum Health: A Proven Progra...        Andrew Weil  
1149777   The Sherbrooke Bride (Bride Trilogy (Paperback))  Catherine Coulter  
1149778                                  Fourth Grade Rats     Jerry Spinelli  
1149779                                                NaN                NaN  

[1149780 rows x 5 columns]>

In [7]:
df.shape

(1149780, 5)

In [8]:
#Get users who left more than 200 reviews
#Get books with  more than 100
users= df["user"].value_counts()
isbn= df["isbn"].value_counts()

users = users[users >= 200].index
isbn = isbn[isbn >= 100].index



In [9]:
#Make a new data frame
df = df[(df["user"].isin(users)) & (df["isbn"].isin(isbn))]

In [10]:
#Drop duplicate reviews
df = df.drop_duplicates(["title","user"])

In [11]:
df.isna().sum()

,0
user,0
isbn,0
rating,0
title,238
author,238


In [12]:
#Dropping isbns without title and author
df = df.dropna(how='any')

In [13]:
df.shape

(49136, 5)

In [14]:
#Create a pivot table
df_pivot = df.pivot_table(index='title',columns='user',values='rating').fillna(0)
df_pivot.head(3)

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
#Create a matrix
df_matrix = csr_matrix(df_pivot.values)

In [16]:
# Creating and trainig the model
nn = NearestNeighbors(metric='cosine',algorithm='brute')
nn.fit(df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [38]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
       # Create a list to store recommended books
       recommended_books = [book, []]  # Initialize with the input book and an empty list for recommendations

       # Pass through our book
       distance, book_info = nn.kneighbors([df_pivot.loc[book]], 6, return_distance=True)

       # Gather the text and distance & reverse the order
       recom_book_info = df_pivot.iloc[np.flip(book_info[0])[:-1]].index.to_list()
       recom_distance = list(np.flip(distance[0])[:-1])

       # Append recommendations to the list
       for r in zip(recom_book_info,recom_distance):
           recommended_books[1].append(list(r))  # Append to the second element (the list)

       return recommended_books  # Return the list of recommended books


In [28]:
# Test function
get_recommends('The Catcher in the Rye')

In [39]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
